In [ ]:
import sys
sys.path.append('../')

### Run the optimization

In [ ]:
import os

import numpy as np
from common import (evaluate_best_coin, get_clustered_dataframes, objective)

import mlflow
import optuna

os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"
mlflow.set_experiment('Training_LSTM_RNN_Cluster')

# Normalize the data to the range [0, 1] to help the LSTM model converge faster

clusters_data = get_clustered_dataframes()

for cluster, data in clusters_data.items():
    print(f"Cluster {cluster}")
    data = data.drop(columns=['Date'])
    criptos = ','.join(data.columns)
    data = np.array(data)

    study_name = f"Cluster_{cluster}"  # Unique identifier of the study.
    storage_name = "sqlite:///training_cluster.db"
    pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    study = optuna.create_study(study_name=study_name, direction='minimize', storage=storage_name, pruner=pruner)

    # Start the optimization process
    study.optimize(lambda trial: objective(trial, data, criptos), n_trials=50)

    # Get the best hyperparameters
    try:
        best_params = study.best_params
        print("Best Hyperparameters:", best_params)
        evaluate_best_coin(criptos, data, best_params)
    except ValueError:
        print("No best hyperparameters found")